# Collect Datasets
First we look at the discipline page.

## Suspension

In [40]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import os

In [15]:
url = "https://www.cde.ca.gov/ds/ad/filessd.asp"
html = requests.get("https://www.cde.ca.gov/ds/ad/filessd.asp").text
soup = BeautifulSoup(html,'html')

In [22]:
table = soup.find("table")
files = {}
for j,row in enumerate(table.find_all("tr")):
    if j==0: continue #Skip header of the table
    date = None
    file_path = None
    for i,cell in enumerate(row.find_all("td")):
        if i==0:
            #Get first column as the date
            date = cell.text.strip()
        elif i==1:
            file_path = cell.find('a').attrs['href']
    files[date]=file_path
    print(date,file_path)

2020–21 https://www3.cde.ca.gov/demo-downloads/discipline/susp2021.txt
2019–20 https://www3.cde.ca.gov/demo-downloads/discipline/susp1920.txt
2018–19 https://www3.cde.ca.gov/demo-downloads/discipline/susp1819.txt
2017–18 https://www3.cde.ca.gov/demo-downloads/discipline/susp1718.txt
2016–17 https://www3.cde.ca.gov/demo-downloads/discipline/susp1617.txt
2015–16 https://www3.cde.ca.gov/demo-downloads/discipline/susp1516.txt
2014–15 https://www3.cde.ca.gov/demo-downloads/discipline/susp1415.txt
2013–14 https://www3.cde.ca.gov/demo-downloads/discipline/susp1314.txt
2012–13 https://www3.cde.ca.gov/demo-downloads/discipline/susp1213.txt
2011–12 https://www3.cde.ca.gov/demo-downloads/discipline/susp1112.txt


Now download them all

use pandas to put them into dataframes

In [29]:
texts = [requests.get(file_path).text for _,file_path in files.items()]

Turn them to dataframes, I have to build the strucutre myself though... parsing.

In [146]:
text = texts[0]
text[:1000]

'AcademicYear\tAggregateLevel\tCountyCode\tDistrictCode\tSchoolCode\tCountyName\tDistrictName\tSchoolName\tCharterYN\tReportingCategory\tCumulativeEnrollment\tTotal Suspensions\tUnduplicated Count of Students Suspended (Total)\tUnduplicated Count of Students Suspended (Defiance-Only)\tSuspension Rate (Total)\tSuspension Count Violent Incident (Injury)\tSuspension Count Violent Incident (No Injury)\tSuspension Count Weapons Possession\tSuspension Count Illicit Drug-Related\tSuspension Count Defiance-Only\tSuspension Count Other Reasons\r\n2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tGF\t110211\t18\t18\t4\t0.0\t0\t11\t1\t1\t4\t1\r\n2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tGM\t117612\t92\t87\t13\t0.1\t11\t52\t2\t3\t13\t11\r\n2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tGX\t82\t0\t0\t0\t0.0\t0\t0\t0\t0\t0\t0\r\n2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tRA\t62638\t31\t30\t6\t0.0\t1\t18\t0\t0\t6\t6\r\n2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tRB\t20475\t6\t6\t1\t0.0\t0\t5\t0\t0\t1\t0\r\n2020-21\tC\t01\t\t\tAlameda\t

In [147]:
text = text.split('\n')
text[:5]

['AcademicYear\tAggregateLevel\tCountyCode\tDistrictCode\tSchoolCode\tCountyName\tDistrictName\tSchoolName\tCharterYN\tReportingCategory\tCumulativeEnrollment\tTotal Suspensions\tUnduplicated Count of Students Suspended (Total)\tUnduplicated Count of Students Suspended (Defiance-Only)\tSuspension Rate (Total)\tSuspension Count Violent Incident (Injury)\tSuspension Count Violent Incident (No Injury)\tSuspension Count Weapons Possession\tSuspension Count Illicit Drug-Related\tSuspension Count Defiance-Only\tSuspension Count Other Reasons\r',
 '2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tGF\t110211\t18\t18\t4\t0.0\t0\t11\t1\t1\t4\t1\r',
 '2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tGM\t117612\t92\t87\t13\t0.1\t11\t52\t2\t3\t13\t11\r',
 '2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tGX\t82\t0\t0\t0\t0.0\t0\t0\t0\t0\t0\t0\r',
 '2020-21\tC\t01\t\t\tAlameda\t\t\tAll\tRA\t62638\t31\t30\t6\t0.0\t1\t18\t0\t0\t6\t6\r']

In [148]:
def clean_line(line):
    return line.replace("ï»¿","").strip()

In [149]:
text = [clean_line(line).split('\t') for line in text]

In [150]:
line_lengths = set()
for i,line in enumerate(text):
    if i < 2:
        print(i,len(line),line)
    line_lengths.add(len(line))
print(line_lengths)

0 21 ['AcademicYear', 'AggregateLevel', 'CountyCode', 'DistrictCode', 'SchoolCode', 'CountyName', 'DistrictName', 'SchoolName', 'CharterYN', 'ReportingCategory', 'CumulativeEnrollment', 'Total Suspensions', 'Unduplicated Count of Students Suspended (Total)', 'Unduplicated Count of Students Suspended (Defiance-Only)', 'Suspension Rate (Total)', 'Suspension Count Violent Incident (Injury)', 'Suspension Count Violent Incident (No Injury)', 'Suspension Count Weapons Possession', 'Suspension Count Illicit Drug-Related', 'Suspension Count Defiance-Only', 'Suspension Count Other Reasons']
1 21 ['2020-21', 'C', '01', '', '', 'Alameda', '', '', 'All', 'GF', '110211', '18', '18', '4', '0.0', '0', '11', '1', '1', '4', '1']
{1, 21}


In [151]:
df = pd.DataFrame(text[1:],columns=text[0])

In [152]:
df

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,ReportingCategory,...,Total Suspensions,Unduplicated Count of Students Suspended (Total),Unduplicated Count of Students Suspended (Defiance-Only),Suspension Rate (Total),Suspension Count Violent Incident (Injury),Suspension Count Violent Incident (No Injury),Suspension Count Weapons Possession,Suspension Count Illicit Drug-Related,Suspension Count Defiance-Only,Suspension Count Other Reasons
0,2020-21,C,01,,,Alameda,,,All,GF,...,18,18,4,0.0,0,11,1,1,4,1
1,2020-21,C,01,,,Alameda,,,All,GM,...,92,87,13,0.1,11,52,2,3,13,11
2,2020-21,C,01,,,Alameda,,,All,GX,...,0,0,0,0.0,0,0,0,0,0,0
3,2020-21,C,01,,,Alameda,,,All,RA,...,31,30,6,0.0,1,18,0,0,6,6
4,2020-21,C,01,,,Alameda,,,All,RB,...,6,6,1,0.0,0,5,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222803,2020-21,T,00,,,State,,,Yes,SH,...,11,10,0,0.1,1,3,1,3,0,3
222804,2020-21,T,00,,,State,,,Yes,SM,...,4,4,0,0.3,0,2,0,2,0,0
222805,2020-21,T,00,,,State,,,Yes,SS,...,356,318,15,0.1,35,186,19,81,16,19
222806,2020-21,T,00,,,State,,,Yes,TA,...,579,516,25,0.1,63,304,32,127,26,27


# That worked
Okay that worked for one school let me now do it for all of them


In [139]:
#Given the text from the file
#Return an an array of arrays that pandas can read. 
def clean(string):
    lines = string.split('\n')
    return [clean_line(line).split('\t') for line in lines]

In [140]:
dataframes = []
for text in texts:
    data = clean(text)
    dataframes.append(pd.DataFrame(data[1:],columns=data[0]))

In [141]:
df = pd.concat(dataframes)
df

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,ReportingCategory,...,Unduplicated Count of Students Suspended (Total),Unduplicated Count of Students Suspended (Defiance-Only),Suspension Rate (Total),Suspension Count Violent Incident (Injury),Suspension Count Violent Incident (No Injury),Suspension Count Weapons Possession,Suspension Count Illicit Drug-Related,Suspension Count Defiance-Only,Suspension Count Other Reasons,Cumulative Enrollment
0,2020-21,C,01,,,Alameda,,,All,GF,...,18,4,0.0,0,11,1,1,4,1,NaN
1,2020-21,C,01,,,Alameda,,,All,GM,...,87,13,0.1,11,52,2,3,13,11,NaN
2,2020-21,C,01,,,Alameda,,,All,GX,...,0,0,0.0,0,0,0,0,0,0,NaN
3,2020-21,C,01,,,Alameda,,,All,RA,...,30,6,0.0,1,18,0,0,6,6,NaN
4,2020-21,C,01,,,Alameda,,,All,RB,...,6,1,0.0,0,5,0,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345156,2011-12,T,00,,,State,,,No,SS,...,267665,131270,7.4,39743,168137,11589,38440,259383,26603,3611833
345157,2011-12,T,00,,,State,,,No,TA,...,352429,170062,6.0,50553,208868,15014,53127,324745,33503,5890875
345158,2011-12,T,00,,,State,,,All,TA,...,365743,176027,5.8,52086,217052,15446,55369,335079,34670,6285600
345159,2011-12,T,00,,,State,,,Yes,TA,...,14066,6180,2.9,1533,8184,432,2242,10334,1167,490897


In [142]:
df.to_csv("suspension.csv")